# Aula 12

## Autómatos híbridos


*Autómatos híbridos* são modelos de sistemas ciber-físicos. Essencialmente um autómato híbrido é uma *máquina de estados finita*, onde cada estado (designado por *modo de funcionamento*) descreve o comportamento contínuo de um sistema dinâmico modelado por *relações diferenciais ordinárias* (nas variáveis contínuas e nas suas derivadas em relação ao tempo) codificadas num predicado designado por *flow*. Estas variáveis contínuas evoluem num modo de funcionamento enquanto o seu *flow* for válido.
Cada transição discreta entre estados é representada por um arco anotado com um predicado (designado *switch* ou *jump*). Uma transição realiza-se quando o seu *switch* é válido. Ao ocorrer uma transição as variáveis internas dos modos conservam o seu valor, a não ser que lhes seja explicitamente atribuído um novo valor.

Um autómato híbrido pode ser descrito por um FOTS (sobre o qual podemos verificar propriedades lógicas com as metodologias que já estudamos). Nesse processo o FOTS vai *discretizar* as relações diferenciais ordinárias e vai mapea-las num espaço de estados também discreto.




### Exemplo do Termostato

Considere o seguinte autómato híbrido que modela o comportamento de um termostato que controla uma caldeira de aquecimento.

<img src="termostato.png" width="400"/>

Inicialmente a temperatura (aqui representada pela variável $x$) é de 20 graus. Quando a temperatura é menor que 19 graus o termostato pode ligar-se. Como o modo em que o termostato está desligado tem o invariante $x>=18$, esta temperatura estabelece um limite a partir do qual o termostato tem necessariamente que se ligar. O termostato só pode voltar a desligar quando a temperatura excede os 21 graus, fazendo-o necessariamente se esta atingir os 22 graus devido ao invariante no modo em que está ligado.

### Verificação de autómatos híbridos com FOTS

Podemos verificar propriedades de segurança de autómatos híbridos codificando a semântica dos mesmos num FOTS e usando depois os procedimentos de verificação implementados nas aulas anteriores.

A ideia básica é incluir nos estados do FOTS para além das variáveis contínuas do autómato híbrido 2 variáveis especiais:
- $T$ é uma variável contínua que denota o tempo
- $M$ é uma variável discreta que denota o *modo de funcionamento*

O estado inicial do FOTS é derivado facilmente a partir da definição do autómato híbrido. Por exemplo, no caso do termostato temos

$$
T = 0 \wedge M = \mathit{Off} \wedge x = 20
$$

As transições do FOTS incluem os dois tipos de transição que podem ocorrer num autómato híbrido:
- Transições *timed* descrevem os *flows* associados a cada modo (a evolução das variáveis contínuas)
- Transições *untimed* descrevem os *switches* entre modos

As transições *untimed* podem ser obtidas através de uma codificação muito directa das guardas e efeitos especificadas nos *switches*, com a restrição que o tempo não evolui nestas transições, nem as variáveis contínuas se modificam a não ser que lhes seja explicitamente atribuído um novo valor no efeito do *switch*. Por exemplo, no caso do termostato temos 2 transições deste tipo:

$$
\begin{array}{c}
M = \mathit{Off} \wedge M' = \mathit{On} \wedge x < 19 \wedge x' = x \wedge T' = T \\
\vee\\
M = \mathit{On} \wedge M' = \mathit{Off} \wedge x > 21 \wedge x' = x \wedge T' = T 
\end{array}
$$

Nas transições *timed* o modo permanece constante, mas o resto das variáveis evoluem de acordo com as restrições indicadas. Os *flows* são especificados indicando qual a derivada em relação ao tempo de cada variável contínua. Para codificar os *flows* no FOTS é necessário fazer a sua *discretização*, ou seja, indicar qual a variação ocorrida no intervalor de tempo $T'-T$. Se a derivada for uma constante a discretização é trivial. Por exemplo, se $\dot{y} = k$ temos que $y' - y = k(T'-T)$. Se a relação de *flow* é amortecida, como por exemplo no modo $\mathit{Off}$ do termostato, onde temos $\dot{x} = -0.1x$, uma sugestão para discretizar consiste em usar um valor constante inferido a partir do invariante de modo. Por exemplo, no caso do *flow* anterior como sabemos que $x \ge 18$ podemos substituir $x$ por 18 na equação anterior, dando origem à relação $x - x' = -1.8(T'-T)$.
Com esta técnica, no caso do termostato teríamos as seguintes 2 transições *timed* (na segunda aproximamos $x$ por 22, o limite superior da temperatura inferido a partir do invariante do modo $\mathit{On}$):

$$
\begin{array}{c}
M = \mathit{Off} \wedge M' = M \wedge x' - x = -1.8 \cdot (T'-T) \wedge T'>T \\
\vee\\
M = \mathit{On} \wedge M' = M \wedge x' - x = 2.8 \cdot (T'-T) \wedge T'>T
\end{array}
$$

Para reduzir os erros na verificação pode-se reduzir a granularidade da discretização subdividindo cada modo em vários sub-modos que cubram toda a gama dos valores permitidos. Por exemplo neste caso poderíamos dividir cada um dos modos em 4 sub-modos, cada um com uma variação de temperatura máxima de 1 grau, cobrindo assim toda a gama de temperaturas possíveis (entre 18 e 22 graus).

Finalmente, é necessário também impor os invariantes dos modos no FOTS. Isso pode ser feito acrescentando a cada transição uma restrição que obriga o invariante a ser cumprido, ou então, modificando os procedimentos de verificação para impor o seguinte invariante:

$$
\begin{array}{c}
M = \mathit{Off} \rightarrow x \ge 18\\
\wedge\\
M = \mathit{On} \rightarrow x \le 22
\end{array}
$$

Na codificação em Z3 é conveniente usar um tipo enumerado para implementar os modos. No caso do termostato podemos declarar esse tipo da seguinte forma.

In [1]:
from z3 import *

Mode, (on, off) = EnumSort('Mode', ('On', 'Off'))

Podemos agora declarar as variáveis do FOTS correspondente ao termostato da seguinte forma.

In [2]:
def declare(i):
    s = {}
    s['T'] = Real('T'+str(i))
    s['M'] = Const('M'+str(i),Mode)
    s['x'] = Real('x'+str(i))
    return s

### Exercício 1

Codifique os predicados Z3 `init`, `trans` e `inv`, que caracterizam, respectivamente, os estados iniciais, as transições e o invariante de modo do FOTS correspondente ao termostato.

In [3]:
def init(s):
    return And(s['T'] == 0, s['M'] == off, s['x'] == 20)

def trans(s,p):
    tOffOn = And(s['M'] == off, p['M'] == on, s['x'] < 19, p['x'] == s['x'], p['T'] == s['T'])
    tOnOff = And(s['M'] == on, p['M'] == off, s['x'] > 21, p['x'] == s['x'], p['T'] == s['T'])
    tOff   = And(s['M'] == off, p['M'] == s['M'], p['x'] - s['x'] == -1.8 * (p['T'] - s['T']), p['T'] > s['T'])
    tOn    = And(s['M'] == on, p['M'] == s['M'], p['x'] - s['x'] == 2.8 * (p['T'] - s['T']), p['T'] > s['T'])
    
    return Or(tOffOn, tOnOff, tOff, tOn)

def inv(s):
    return And(Implies(s['M'] == off, s['x'] >= 18), Implies(s['M'] == on, s['x'] <= 22))

### Exercício 2

Adapte a função `gera_traco` implementada nas aulas anterior para receber também como parâmetro o invariante de modo. Para melhor compreensão dos resultados, imprima também todas as variáveis continuas como números de virgula flutuante. Para saber qual o tipo (*sort*) de uma variável use o método `sort()`. No caso das variáveis contínuas o tipo será `RealSort()`. Para converter o valor `v` de uma variável contínua do Z3 para um float do Python use a expressão `float(v.numerator_as_long())/float(v.denominator_as_long())`.

In [4]:
def gera_traco(declare,init,trans,inv,k):
    s = Solver()
    
    # criar k cópias de estado, guardar na lista do traço
    trace = []
    for i in range(k):
        trace.append(declare(i))
        
    s.add(init(trace[0]))
    
    for i in range(k):
        s.add(inv(trace[i]))
    
    for i in range(k - 1):
        s.add(trans(trace[i], trace[i + 1]))
    
    if s.check() == sat:
        m = s.model()
        for i in range(k):
            print(i)
            for v in trace[i]:
                if trace[i][v].sort() != RealSort():
                    print(v, '=', m[trace[i][v]])
                else:
                    r = m[trace[i][v]]
                    f = float(r.numerator_as_long())/float(r.denominator_as_long())
                    print(v, '=', f)
            
                
gera_traco(declare,init,trans,inv,5)

0
T = 0.0
M = Off
x = 20.0
1
T = 0.37037037037037035
M = Off
x = 19.333333333333332
2
T = 0.7407407407407407
M = Off
x = 18.666666666666668
3
T = 1.1111111111111112
M = Off
x = 18.0
4
T = 1.1111111111111112
M = On
x = 18.0


### Exercício 3

Adapte a função `bmc_always` implementada nas aulas anterior para receber também como parâmetro o invariante de modo `inv` na verificação da propriedade `prop`. Tal como no exercício anterior, quando ocorrer um contra-exemplo imprima também todas as variáveis continuas como números de virgula flutuante. 

In [5]:
def bmc_always(declare,init,trans,inv,prop,K):
    s = Solver()
    
    # criar k cópias de estado, guardar na lista do traço
    trace = []
    for i in range(K):
        trace.append(declare(i))
        
    s.add(init(trace[0]))
    
    for i in range(K):
        s.add(inv(trace[i]))
    
    for i in range(K - 1):
        s.add(trans(trace[i], trace[i + 1]))
        
    s.add(Not(prop(trace[K - 1])))
    
    if s.check() == sat:
        m = s.model()
        for i in range(K):
            print(i)
            for v in trace[i]:
                if trace[i][v].sort() != RealSort():
                    print(v, '=', m[trace[i][v]])
                else:
                    r = m[trace[i][v]]
                    f = float(r.numerator_as_long())/float(r.denominator_as_long())
                    print(v, '=', f)
        return
    
    print ("A propriedade é válida de em traços de tamanho até " + str(K))

def positive(s):
    return s['x'] >= 0

bmc_always(declare,init,trans,inv,positive,10)

A propriedade é válida de em traços de tamanho até 10


### Exercício 4

Verifique que não é verdade que a temperatura seja sempre inferior a 22 graus.

In [6]:
def never22(s):
    return s['x'] < 22

bmc_always(declare,init,trans,inv,never22,10)

0
T = 0.0
M = Off
x = 20.0
1
T = 1.1111111111111112
M = Off
x = 18.0
2
T = 1.1111111111111112
M = On
x = 18.0
3
T = 1.6666666666666667
M = On
x = 19.555555555555557
4
T = 2.380952380952381
M = On
x = 21.555555555555557
5
T = 2.380952380952381
M = Off
x = 21.555555555555557
6
T = 2.9365079365079363
M = Off
x = 20.555555555555557
7
T = 4.109347442680776
M = Off
x = 18.444444444444443
8
T = 4.109347442680776
M = On
x = 18.444444444444443
9
T = 5.379188712522046
M = On
x = 22.0


### Exercício 5

A propriedade "o termostato irá inevitavelmente ficar ligado" é uma propriedade de *liveness* exprimível como $F\ (M = \mathit{On})$. No entanto a propriedade "passados 1.1s o termostato irá inevitavelmente ficar ligado" já é uma propriedade de *safety*. Explique porquê. Verifique também se esta propriedade se verifica no termostato.

In [7]:
def on11(s):
    return Implies(s['T']==1, s['M']==on)
    
bmc_always(declare, init, trans, inv, on11, 10)

0
T = 0.0
M = Off
x = 20.0
1
T = 0.6049382716049383
M = Off
x = 18.91111111111111
2
T = 0.654320987654321
M = Off
x = 18.822222222222223
3
T = 0.7037037037037037
M = Off
x = 18.733333333333334
4
T = 0.7530864197530864
M = Off
x = 18.644444444444446
5
T = 0.8024691358024691
M = Off
x = 18.555555555555557
6
T = 0.8518518518518519
M = Off
x = 18.466666666666665
7
T = 0.9012345679012346
M = Off
x = 18.377777777777776
8
T = 0.9506172839506173
M = Off
x = 18.288888888888888
9
T = 1.0
M = Off
x = 18.2


### Exercício 6

Como pode verificar o contra-exemplo obtido não é realista e deve-se aos erros introduzidos pela fraca discretização efectuada. Neste exercício pretende-se que reduza a granularidade da discretização para 1 grau e que volte a verificar esta propriedade. Para tal teremos que dividir os 2 modos em 4 sub-modos cada correspondentes às 4 gamas de temperatura possíveis: $18 \le x \le 19$, $19 \le x \le 20$, $20 \le x \le 21$, e $21 \le x \le 22$. Sugere-se que, em vez de alterar o tipo da variável $M$, introduza uma nova variável de modo discreta $D \in \{18,19,20,21\}$ que indica o limite inferior de cada gama de temperatura. Com esta nova variável, o invariante de modo pode ser descrito como

$$18 \le D \le 21 \wedge D \le x \le D+1$$

Para converter uma variável discreta `i` para uma variável contínua pode usar a função `ToReal(i)`.

In [8]:
# completar